Integrantes
   - Amanda Lima
   - Camila Carraro
   - Dylan Ricardo
   - Matheus Zanellato

# 1. Materiais e Metodos

*   Base e pré-processamento
*   Descritores de Características
*   Modelos

## Importando as bibliotecas e carregando a base

In [1]:
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

import seaborn as sns
import pandas as pd

from google.colab.patches import cv2_imshow # for image display
import skimage.feature as feature

In [ ]:
!wget https://github.com/Amandals/ComputerVision/blob/development/basesimpsons.zip?raw=true -O dataset.zip 

!wget https://github.com/Amandals/ComputerVision/blob/development/Teste.zip?raw=true -O teste.zip 
!wget https://github.com/Amandals/ComputerVision/blob/development/Treino.zip?raw=true -O treino.zip 

!unzip -o dataset.zip
!unzip -o teste.zip -d Teste
!unzip -o Treino.zip -d Treino

In [ ]:
train_list = glob.glob('/content/Treino/*.bmp')
print(train_list)

In [ ]:
test_list = glob.glob('/content/Teste/*.bmp')
print(test_list)

In [5]:
def generate_labels(data_list):
  result = []
  for name in data_list:
    if name.find('bart') != -1:
      result.append(0)
    elif name.find('homer') != -1:
      result.append(1)
    elif name.find('lisa') != -1:
      result.append(2)
    elif name.find('marge') != -1:
      result.append(3)
    elif name.find('maggie') != -1:
      result.append(4)
    elif name.find('family') != -1:
      result.append(5)
  return result

In [6]:
train_labels = generate_labels(train_list)
test_labels = generate_labels(test_list)

In [7]:
#Auxiliary Function to plot side by side
def plot_sidebyside(img_list,titles,colormap=None,figsize=(12,6)):
  n = len(img_list)
  figure, axis = plt.subplots(1, n, figsize=figsize)

  for i in range(n):
    axis[i].imshow(img_list[i], cmap=colormap)
    axis[i].set_title(titles[i])
    axis[i].axis('off')
  # Combine all the operations and display
  plt.show()

## Pre-Processamento

### Tamanho

Normalizacao de tamanhos, foram definidos diferentes "ranges" de tamanhos para aumentar ou diminuir as imagens. Esses ranges preservam detalhes das imagens por evitar reducoes ou aumentos muito drasticos.

In [10]:
def resize_data(im_list):
  result_list = []
  for im_name in im_list:
    im = cv2.imread(im_name)
    s = im.shape

    objective = 350
    if s[0] < 150:
      percent = s[0]/objective + 1
      new_height = int(s[0] + (s[0] * percent))
      new_width = int(s[1] + (s[1] * percent))
      im = cv2.resize(im, (new_width, new_height))
 
    elif s[0] >= 150 and s[0] <= 250:
      percent = s[0]/objective
      new_height = int(s[0] + (s[0] * percent))
      new_width = int(s[1] + (s[1] * percent))
      im = cv2.resize(im, (new_width, new_height))
    
    elif s[0] > 450 and s[0] <= 530:
      percent = objective/s[0] - 0.3
      new_height = int(s[0] - (s[0] * percent))
      new_width = int(s[1] - (s[1] * percent))
      im = cv2.resize(im, (new_width, new_height))

    elif s[0] > 530 and s[0] <= 650:
      percent = objective/s[0] - 0.2
      new_height = int(s[0] - (s[0] * percent))
      new_width = int(s[1] - (s[1] * percent))
      im = cv2.resize(im, (new_width, new_height))

    elif s[0] > 650:
      percent = objective/s[0] + 0.1
      new_height = int(s[0] - (s[0] * percent))
      new_width = int(s[1] - (s[1] * percent))
      im = cv2.resize(im, (new_width, new_height))

    result_list.append(im)
  return result_list

In [11]:
train_resized = resize_data(train_list)

In [12]:
test_resized = resize_data(test_list)

In [13]:
# for im in train_resized:
#   cv2_imshow(im)

### Cor

In [ ]:
for im in train_resized:
  cv2_imshow(im)
  # Plot dos canais de cor
  ch_b = im[:,:,0]
  ch_g = im[:,:,1]
  ch_r = im[:,:,2]

  plot_sidebyside([ch_b,ch_g,ch_r],["Blue Channel", "Green Channel", "Red Channel"])

In [ ]:
for im in train_resized:
  # Plot da imagem original
  cv2_imshow(im)
  hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

  # plt.title("HSV")
  # plt.imshow(hsv,cmap='hsv')
  # plt.show()

  H = hsv[:,:,0]
  S = hsv[:,:,1]
  V = hsv[:,:,2]
  h_s_v = [H, S, V]
  # plot_sidebyside([im,H,S,V],["Original", "Hue Channel","Saturaturation Channel","Value Channel"])
  
  val = 200
  for i in h_s_v:
    _, thresh = cv2.threshold(i, val, 255, cv2.THRESH_BINARY_INV)
    plt.title("Thresholding Hue Channel: " + str(val))
    plt.imshow(thresh, cmap='gray')
    plt.show()

Apos a analise dos campos de cor e hsv podemos notar que o campo "value" eh capaz de ressaltar as caracteristicas dos cabelos, olhos e corpos dos personagens, parece um bom caminho a explorar, apesar de isso nao considerar as cores marcantes de cada personagem. A escolha de relevar as cores aqui pode ser prejudicial na extracao das features, caso aconteca testes com as cores tambem serao feitos

#### Explorando threshold
agora eh necessario definir o melhor threshold para o canal "Value" 

In [ ]:
for im in train_resized:
  # cv2_imshow(im)
  hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

  V = hsv[:,:,2]

  vals = [200, 220, 230, 240, 250, 252, 253, 254]
  thresh_list = [im]
  for v in vals:
    _, thresh = cv2.threshold(V, v, 255, cv2.THRESH_BINARY_INV)
    thresh_list.append(thresh)

  plot_sidebyside(thresh_list, ["Original", "TVC: 200", "TVC: 220", "TVC: 230", "TVC: 240", "TVC: 250", "TVC: 252", "TVC: 253", "TVC: 254"], colormap='gray')

um threshold de 230 parece separar os componentes mais importantes dos personagens

In [17]:
def threshold_data(im_list):
  result_list = []
  for im in im_list:
    V = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)[:,:,2]
    _, thresh = cv2.threshold(V, 230, 255, cv2.THRESH_BINARY_INV)
    result_list.append(thresh)
  return result_list

In [18]:
train_val_thresh = threshold_data(train_resized)
test_val_thresh = threshold_data(test_resized) 

In [19]:
# for im in train_val_thresh:
#   cv2_imshow(im)

## Descritores "Edge descriptors"

In [ ]:
def extract_edges(data, algorithm='canny'):
  features = []

  for img in data:
    #Evaluate smooth on fashion mnist data
    #img = cv2.GaussianBlur(img,(5,5),0)
    #_,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

    if algorithm=='canny':
      edges = cv2.Canny(img, 100, 200)      
    
    elif algorithm=='sobel':
      edges = cv2.Sobel(img, cv2.CV_64F, 1, 1)
      #Back to UINT8
      edges = cv2.convertScaleAbs(edges) 

    elif algorithm=='laplace':
      edges = cv2.Laplacian(img, cv2.CV_64F)
      #Back to UINT8
      edges = cv2.convertScaleAbs(edges)
    
    features.append(edges)

  return np.array(features)

train_feat_canny = extract_edges(train_val_thresh, 'canny')
test_feat_canny = extract_edges(test_val_thresh, 'canny')

train_feat_sobel = extract_edges(train_val_thresh, 'sobel')
test_feat_sobel = extract_edges(test_val_thresh, 'sobel')

train_feat_laplace = extract_edges(train_val_thresh, 'laplace')
test_feat_laplace = extract_edges(test_val_thresh, 'laplace')

for canny,sobel,laplace,img in zip(train_feat_canny, train_feat_sobel, train_feat_laplace, train_val_thresh): 
  plot_sidebyside([img, canny, sobel, laplace],['img', 'canny', 'sobel', 'laplace'], 'gray', figsize=(10,6))

dos descritores analisados acima o que destaca as caracteristicas mais importantes na maioria dos casos eh o que usa o algoritmo de laplace

## Descritores "Shape descriptors"

#### HU

In [21]:
class HuMoments:
  def describe(self,im,threshold=128):

    if len(im.shape) > 2 and im.shape[2] == 3:
      im = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
    

    _,threshold = cv2.threshold(im, threshold, 255, cv2.THRESH_BINARY)

    # Calculate Moments 
    moments = cv2.moments(threshold) 
    # Calculate Hu Moments 
    huMoments = cv2.HuMoments(moments)

    # Log scale hu moments 
    for i in range(0,len(huMoments)):
      if huMoments[i] != 0:        
        huMoments[i] = -1* math.copysign(1.0, huMoments[i]) * math.log10(abs(huMoments[i]))

    #There is resulting image for HuMoments
    return huMoments.reshape(huMoments.shape[0]), threshold

In [22]:
def humoments_data(data_list):
  hum = HuMoments()
  result_img = []
  result_feat = []

  for im in data_list:
    feat, res_img = hum.describe(im)
    result_feat.append(feat)
    result_img.append(res_img)

  result_feat = np.reshape(result_feat, (len(data_list), -1))
  return result_img, result_feat

#### HOG

In [23]:
class HOG:
  def describe(self,im,pixels_per_cell=(8, 8),cells_per_block=(2, 2)):
    if len(im.shape) > 2 and im.shape[2] == 3:
      im = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)

    im = cv2.resize(im,(64,128))

    fd, hog_image = feature.hog(im, orientations=9, pixels_per_cell=pixels_per_cell,
                	cells_per_block=(2, 2),visualize=True)
    return fd,hog_image

In [24]:
def hog_data(data_list):
  hog = HOG()
  result_img = []
  result_feat = []

  for im in data_list:
    feat, res_img = hog.describe(im)
    result_feat.append(feat)
    result_img.append(res_img)

  result_feat = np.reshape(result_feat, (len(data_list), -1))
  return result_img, result_feat

### Tests

A partir do descritor com laplce decidimos usar as duas estrategias abaixo para classificar com KNN e SVM, portanto sao gerados as listas com as imagens e as features tanto para HUM quanto para HOG.

In [ ]:
hum = HuMoments()
for im in train_feat_laplace:
  _, res_img = hum.describe(im)
    
  plot_sidebyside([im, res_img], ["Input", "Hu Moments"], colormap='gray')

In [26]:
train_laplace_hum, train_features_hum = humoments_data(train_feat_laplace)
test_laplace_hum, test_features_hum = humoments_data(test_feat_laplace)

In [ ]:
train_laplace_HOG = []
hog = HOG()
for im in train_feat_laplace:
  _, res_img = hog.describe(im)
    
  plot_sidebyside([im, res_img], ["Input", "HOG"], colormap='gray')

In [28]:
train_laplace_hog, train_features_hog = hog_data(train_feat_laplace)
test_laplace_hog, test_features_hog = hog_data(test_feat_laplace)

# 2. Experimentos: Detalhamento do treinamento dos modelos escolhidos e exposição dos resultados

* Treinamento dos modelos
* Matriz de Confusão e Taxas de Acerto

In [29]:
# {'bart':0, 'homer':1, 'lisa':2, 'marge': 3, 'maggie' : 4, 'family':5} 

In [30]:
def performance_evaluation(x_test, y_test, predictions, info_message):
  
  print(f"Evaluation of ", info_message)
  print(f"{metrics.classification_report(y_test, predicted)}\n")
  disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predictions)
  disp.figure_.suptitle("Confusion Matrix")
  plt.show()


In [31]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

## KNN

### KNN HUM

In [ ]:
train = train_features_hum
test = test_features_hum
print('Processing - KNN - Hu Moments')
print(train.shape, test.shape) 

knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(train, train_labels)
predicted = knn.predict(test)
performance_evaluation(test, test_labels, predicted, 'Hu Moments')

### KNN HOG

In [ ]:
train = train_features_hog
test = test_features_hog
print('Processing - HOG')
print(train.shape, test.shape) 

knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(train, train_labels)
predicted = knn.predict(test)
performance_evaluation(test, test_labels, predicted, 'HOG')

## SVM

### SVM HUM

In [ ]:
train = train_features_hum
test = test_features_hum
print('Processing - SVM - Hu Moments')
print(train.shape, test.shape) 

svm = SVC()
svm.fit(train, train_labels)
predicted = svm.predict(test)
performance_evaluation(test, test_labels, predicted, 'SVM - Hu Moments')

### SVM HOG

In [ ]:
train = train_features_hog
test = test_features_hog
print('Processing - SVM - HOG')
print(train.shape, test.shape) 

svm = SVC()
svm.fit(train, train_labels)
predicted = svm.predict(test)
performance_evaluation(test, test_labels, predicted, 'SVM - HOG')

## RF

### RF HUM

In [ ]:
train = train_features_hum
test = test_features_hum
print('Processing - Random Forest - Hu Moments')
print(train.shape, test.shape) 

rf = RandomForestClassifier(n_estimators=340, random_state=1, min_samples_split=10)
rf.fit(train, train_labels)
predicted = rf.predict(test)
performance_evaluation(test, test_labels, predicted, 'Random Forest - Hu Moments')

### RF HOG

In [ ]:
train = train_features_hog
test = test_features_hog
print('Processing - Random Forest - HOG')
print(train.shape, test.shape) 

rf = RandomForestClassifier(n_estimators=250, random_state=1, min_samples_split=5)
rf.fit(train, train_labels)
predicted = rf.predict(test)
performance_evaluation(test, test_labels, predicted, 'Random Forest - HOG')

## ADA BOOST

### ADA HUM

In [ ]:
train = train_features_hum
test = test_features_hum
print('Processing - Ada Boost - Hu Moments')
print(train.shape, test.shape) 

classif = DecisionTreeClassifier(max_depth=5, random_state=1)
ada = AdaBoostClassifier(base_estimator=classif)
ada.fit(train, train_labels)
predicted = ada.predict(test)
performance_evaluation(test, test_labels, predicted, 'Ada Boost - Hu Moments')

### ADA HOG

In [ ]:
train = train_features_hog
test = test_features_hog
print('Processing - Ada Boost - HOG')
print(train.shape, test.shape) 

classif = LogisticRegression(penalty='none', random_state=1)
ada = AdaBoostClassifier(base_estimator=classif)
ada.fit(train, train_labels)
predicted = ada.predict(test)
performance_evaluation(test, test_labels, predicted, 'Ada Boost - HOG')

# 3. Discussão:

* Análise e Discussão Crítica dos resultados
* Expor casos de acerto e erros
* OBS: Convença o avaliador sobre o seu entendimento sobre o que foi implementado, porque foi implementado, uma análise embasada dos resultados positivos e negativos, tal qual discutido em sala de aula
